In [1]:
"""
After following the steps in Image_Analysys.ipynb and extracting features from all of the
(including test/train/test_masked)
We will train our KNN model
"""

'\nAfter following the steps in Image_Analysys.ipynb and extracting features from all of the\n(including test/train/test_masked)\nWe will train our KNN model\n'

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [6]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.neighbors

#Reading training and test data
train_df_org = pd.read_csv("train_final_v2.csv")
test_df_org = pd.read_csv("test_final_v2.csv")
submission_org = pd.read_csv("test_masked_v2.csv")
feature_cols = [
    'NDVI', 'AWEI', 'NDWI', 'MNDWI', 'BLUE',
    'GREEN', 'RED', 'NIR', 'SWIR_1', 'THERMAL', 'SWIR_2', 'PANCH',
    'lat', 'lon', 'n_asset', 'asset_index'
]

label_cols = ['water_index']

#Cleaning the data and replacing Nans with the median of that column
for c in feature_cols:
    test_df_org[c] = test_df_org[c].replace(np.NaN, test_df_org[c].median())
    train_df_org[c] = train_df_org[c].replace(np.NaN, train_df_org[c].median())
    submission_org[c] = submission_org[c].replace(np.NaN, submission_org[c].median())
    
train_df = train_df_org[(feature_cols + label_cols)]
test_df = test_df_org[(feature_cols + label_cols)]

#Seperating X, y for train/test and submission
X_train = train_df[feature_cols]
y_train = train_df[label_cols]
X_test = test_df[feature_cols]
y_test = test_df[label_cols]
X_sub = submission_org[feature_cols]


#Merging train+test to re-split them and try different ratios
X_total = pd.concat((X_test, X_train))
y_total = pd.concat((y_test, y_train))

test_ratio = 0.3

X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=test_ratio, random_state=42)

#Scaling the X values
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Trying different configurations:
"""
Config 1: use scaled X + set weights to distance
Config 2: use scaled X + set weights to uniform
Config 3: use non-scaled X + set weights to distance
Config 4: use non-scaled X + set weights to uniform
"""

min_mse = 1
config = ""

for k in range(5, 50):
    knn = sklearn.neighbors.KNeighborsRegressor(n_neighbors=k, weights='distance', algorithm='auto')
    knn.fit(X_train_scaled, y_train)
    preds = knn.predict(X_test_scaled)
    mse = sklearn.metrics.mean_squared_error(y_test, preds)
    print(f'k={k} - distance - {mse}')
    
    if mse < min_mse:
        min_mse = mse
        config = f'k={k} - distance - {mse}'
    
    knn = sklearn.neighbors.KNeighborsRegressor(n_neighbors=k, weights='uniform', algorithm='auto')
    knn.fit(X_train_scaled, y_train)
    preds = knn.predict(X_test_scaled)
    mse = sklearn.metrics.mean_squared_error(y_test, preds)
    print(f'k={k} - uniform - {mse}')
    
    if mse < min_mse:
        min_mse = mse
        config = f'k={k} - uniform - {mse}'
    
    knn = sklearn.neighbors.KNeighborsRegressor(n_neighbors=k, weights='distance', algorithm='auto')
    knn.fit(X_train, y_train)
    preds = knn.predict(X_test)
    mse = sklearn.metrics.mean_squared_error(y_test, preds)
    print(f'k={k} - NS - distance - {mse}')
    
    if mse < min_mse:
        min_mse = mse
        config = f'k={k} - NS - distance - {mse}'
    
    knn = sklearn.neighbors.KNeighborsRegressor(n_neighbors=k, weights='uniform', algorithm='auto')
    knn.fit(X_train, y_train)
    preds = knn.predict(X_test)
    mse = sklearn.metrics.mean_squared_error(y_test, preds)
    print(f'k={k} - NS - uniform - {mse}')
    
    if mse < min_mse:
        min_mse = mse
        config = f'k={k} - NS - uniform - {mse}'
    

k=5 - distance - 0.3120029315329122
k=5 - uniform - 0.316218244305744
k=5 - NS - distance - 0.24074297975767858
k=5 - NS - uniform - 0.2437921796409563
k=6 - distance - 0.304668606109504
k=6 - uniform - 0.3085551886112367
k=6 - NS - distance - 0.23778340748628252
k=6 - NS - uniform - 0.2413556801428338
k=7 - distance - 0.30173210820810314
k=7 - uniform - 0.3060958963013175
k=7 - NS - distance - 0.23472922611978733
k=7 - NS - uniform - 0.23839225762616031
k=8 - distance - 0.2985231156696616
k=8 - uniform - 0.30290644724153193
k=8 - NS - distance - 0.23464476043208032
k=8 - NS - uniform - 0.23891530877242764
k=9 - distance - 0.29815128766737575
k=9 - uniform - 0.3029079589324749
k=9 - NS - distance - 0.23198272468153527
k=9 - NS - uniform - 0.23639759761163254
k=10 - distance - 0.2983742365238372
k=10 - uniform - 0.3035404062077291
k=10 - NS - distance - 0.23034519138201529
k=10 - NS - uniform - 0.2343758264169488
k=11 - distance - 0.29858191799285455
k=11 - uniform - 0.303994204533005
k

In [7]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.neighbors

#Reading training and test data
train_df_org = pd.read_csv("train_final_v2.csv")
test_df_org = pd.read_csv("test_final_v2.csv")
submission_org = pd.read_csv("test_masked_v2.csv")
feature_cols = [
    'NDVI', 'AWEI', 'NDWI', 'MNDWI', 'BLUE',
    'GREEN', 'RED', 'NIR', 'SWIR_1', 'THERMAL', 'SWIR_2', 'PANCH',
    'lat', 'lon', 'n_asset', 'asset_index'
]

label_cols = ['water_index']

#Cleaning the data and replacing Nans with the median of that column
for c in feature_cols:
    test_df_org[c] = test_df_org[c].replace(np.NaN, test_df_org[c].median())
    train_df_org[c] = train_df_org[c].replace(np.NaN, train_df_org[c].median())
    submission_org[c] = submission_org[c].replace(np.NaN, submission_org[c].median())
    
train_df = train_df_org[(feature_cols + label_cols)]
test_df = test_df_org[(feature_cols + label_cols)]

#Seperating X, y for train/test and submission
X_train = train_df[feature_cols]
y_train = train_df[label_cols]
X_test = test_df[feature_cols]
y_test = test_df[label_cols]
X_sub = submission_org[feature_cols]


#Merging train+test to re-split them and try different ratios
X_total = pd.concat((X_test, X_train))
y_total = pd.concat((y_test, y_train))

test_ratio = 0.3

X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=test_ratio, random_state=42)

#Scaling the X values
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Trying different configurations:
"""
Config 1: use scaled X + set weights to distance
Config 2: use scaled X + set weights to uniform
Config 3: use non-scaled X + set weights to distance
Config 4: use non-scaled X + set weights to uniform
"""

min_mse = 1
config = ""

for k in range(5, 50):
    knn = sklearn.neighbors.KNeighborsRegressor(n_neighbors=k, weights='distance', algorithm='auto')
    knn.fit(X_train_scaled, y_train)
    preds = knn.predict(X_test_scaled)
    mse = sklearn.metrics.mean_squared_error(y_test, preds)
    print(f'k={k} - distance - {mse}')
    
    if mse < min_mse:
        min_mse = mse
        config = f'k={k} - distance - {mse}'
    
    knn = sklearn.neighbors.KNeighborsRegressor(n_neighbors=k, weights='uniform', algorithm='auto')
    knn.fit(X_train_scaled, y_train)
    preds = knn.predict(X_test_scaled)
    mse = sklearn.metrics.mean_squared_error(y_test, preds)
    print(f'k={k} - uniform - {mse}')
    
    if mse < min_mse:
        min_mse = mse
        config = f'k={k} - uniform - {mse}'
    
    knn = sklearn.neighbors.KNeighborsRegressor(n_neighbors=k, weights='distance', algorithm='auto')
    knn.fit(X_train, y_train)
    preds = knn.predict(X_test)
    mse = sklearn.metrics.mean_squared_error(y_test, preds)
    print(f'k={k} - NS - distance - {mse}')
    
    if mse < min_mse:
        min_mse = mse
        config = f'k={k} - NS - distance - {mse}'
    
    knn = sklearn.neighbors.KNeighborsRegressor(n_neighbors=k, weights='uniform', algorithm='auto')
    knn.fit(X_train, y_train)
    preds = knn.predict(X_test)
    mse = sklearn.metrics.mean_squared_error(y_test, preds)
    print(f'k={k} - NS - uniform - {mse}')
    
    if mse < min_mse:
        min_mse = mse
        config = f'k={k} - NS - uniform - {mse}'
        
"""
After trying different configurations, the optimal(lowest MSE) configuration was:
k=14 - Not Scaled - weights=distance - MSE=0.22897112891895058
"""

#Now fit knn using X_total+y_total(test and train combined) for best k
k = 14
knn = sklearn.neighbors.KNeighborsRegressor(n_neighbors=k, weights='distance', algorithm='auto')
knn.fit(X_total, y_total)
preds = knn.predict(X_test)
mse = sklearn.metrics.mean_squared_error(y_test, preds)

#Predicting and building submission file
X_sub = submission_org[feature_cols]
solution_df = submission_org.copy()
solution_df["water_index"] = knn.predict(X_sub)
filtered_solution_df = solution_df[["DHSID_EA", "water_index"]]

In [8]:
#Predicting and building submission file
X_sub = submission_org[feature_cols]
solution_df = submission_org.copy()
solution_df["water_index"] = knn.predict(X_sub)
filtered_solution_df = solution_df[["DHSID_EA", "water_index"]]
filtered_solution_df.to_csv("solution_10.csv")

In [14]:
filtered_solution_df

,DHSID_EA,water_index
0,IA-2015-7-00010009,5.000000
1,IA-2015-7-00010011,4.957400
2,IA-2015-7-00010017,4.710308
3,IA-2015-7-00010044,4.965015
4,IA-2015-7-00010060,4.992000
...,...,...
2670,IA-2015-7-00360403,4.935505
2671,IA-2015-7-00360454,4.900143
2672,IA-2015-7-00360474,4.914671
2673,IA-2015-7-00360476,4.243696


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
import sklearn.neighbors

#Reading training and test data
train_df_org = pd.read_csv("train_final_v2.csv")
test_df_org = pd.read_csv("test_final_v2.csv")
submission_org = pd.read_csv("test_masked_v2.csv")
feature_cols = [
    'NDVI', 'AWEI', 'NDWI', 'MNDWI', 'BLUE',
    'GREEN', 'RED', 'NIR', 'SWIR_1', 'THERMAL', 'SWIR_2', 'PANCH',
    'lat', 'lon', 'n_asset', 'asset_index'
]

label_cols = ['water_index']

#Cleaning the data and replacing Nans with the median of that column
for c in feature_cols:
    test_df_org[c] = test_df_org[c].replace(np.NaN, test_df_org[c].median())
    train_df_org[c] = train_df_org[c].replace(np.NaN, train_df_org[c].median())
    submission_org[c] = submission_org[c].replace(np.NaN, submission_org[c].median())
    
train_df = train_df_org[(feature_cols + label_cols)]
test_df = test_df_org[(feature_cols + label_cols)]

#Seperating X, y for train/test and submission
X_train = train_df[feature_cols]
y_train = train_df[label_cols]
X_test = test_df[feature_cols]
y_test = test_df[label_cols]
X_sub = submission_org[feature_cols]

#Merging train+test to re-split them and try different ratios
X_total = pd.concat((X_test, X_train))
y_total = pd.concat((y_test, y_train))

test_ratio = 0.3

X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=test_ratio, random_state=42)

#Scaling the X values
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Trying different hyperparameters using cross-validation
configs = []
for k in range(5, 50):
    for w in ['uniform', 'distance']:
        for s in [True, False]:
            knn = sklearn.neighbors.KNeighborsRegressor(n_neighbors=k, weights=w, algorithm='auto', n_jobs=-1)
            scores = cross_val_score(knn, X_train_scaled if s else X_train, y_train, cv=5, scoring='neg_mean_squared_error')
            mean_mse = -1 * np.mean(scores)
            configs.append((k, w, s, mean_mse))

#Finding the best configuration
best_config = min(configs, key=lambda x: x[3])
print(f'Best config: {best_config}')

#Training the best model on the full training set
knn = sklearn.neighbors.KNeighborsRegressor(n_neighbors=best_config[0], weights=best_config[1], algorithm='auto', n_jobs=-1)
knn.fit(X_train_scaled if best_config[2] else X_train, y_train)

#Generating predictions on the test set
X_test_scaled = scaler.transform(X_test)
X_sub_scaled = scaler.transform(X_sub)
test_preds = knn.predict(X_test_scaled)
test_mse = sklearn.metrics.mean_squared_error(y_test, test_preds)
print
